<a href="https://colab.research.google.com/github/sayarghoshroy/Hate-Speech-Detection/blob/master/tweet_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xlrd
import re
import pickle

In [2]:
# Uncomment if you're running it for the first time
# !pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
# to leverage word statistics from Twitter
seg_tw = Segmenter(corpus = "twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [3]:
# Uncomment if you're running it for the first time
# !pip install tweet-preprocessor
import preprocessor as tweet_proc

In [4]:
# Uncomment if you're running it for the first time
# !pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

#### *Raw Datasets are hosted [here](https://drive.google.com/drive/folders/1TuHRQQ41lK9oXJhlhsRiMjMnczuKn_kF?usp=sharing).*

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [6]:
def make_list(proc_obj):
  if proc_obj == None:
    return []
  
  store = []
  for unit in proc_obj:
    store.append(unit.match)
  
  return store

def emotext(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()))
    return text

In [7]:
# For 2020 Datasets

is_hindi = 0

# For English
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/english.xlsx"

# For Hindi
file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/hindi.xlsx"
is_hindi = 1

# For German
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/german.xlsx"

In [8]:
# Initializing Lists
datapoints_count = 0
see_index = True

tweets = []
raw_tweet_texts = []
tokenized_tweets = []
hashtags = []
smileys = []
emojis = []
urls = []
mentions = []
numbers = []
reserveds = []

task_1_labels = []
task_2_labels = []
tweet_ids = []
hasoc_ID = []

In [9]:
def strip_list(listie):
  stripped = []
  for item in listie:
    stripped.append(item.strip())
  return stripped

def hindi_clean(line, parse_obj):
  # beta
  tokens = line.replace(":", " : ").replace(",", " , ").replace(";", " ; ").split(" ")
  valid_stri = ""

  for raw_token in tokens:
    token = raw_token.strip()
    if token in strip_list(make_list(parse_obj.hashtags)):
      continue
    if token in strip_list(make_list(parse_obj.smileys)):
      continue
    if token in strip_list(make_list(parse_obj.emojis)):
      continue
    if token in strip_list(make_list(parse_obj.urls)):
      continue
    if token in strip_list(make_list(parse_obj.mentions)):
      continue
    if token in strip_list(make_list(parse_obj.numbers)):
      continue
    if token in strip_list(make_list(parse_obj.reserved)):
      continue
    valid_stri = valid_stri + " " + token
  return valid_stri.strip()

In [10]:
workbook = xlrd.open_workbook(file_name)
sheet = workbook.sheet_by_index(0)

count = 0
for row in range(sheet.nrows):
    line = sheet.row_values(row)

    if see_index == True:
        see_index = False
        continue

    datapoints_count += 1
    tweet_ids.append(line[0])
    task_1_labels.append(line[2])
    task_2_labels.append(line[3])
    hasoc_ID.append(line[4])
    tweets.append(line[1].replace("\n", " "))

    parse_obj = tweet_proc.parse(line[1].replace("\n", " "))
    tokenized_tweets.append(tweet_proc.tokenize(line[1].replace("\n", " ")))
    hashtags.append(strip_list(make_list(parse_obj.hashtags)))
    smileys.append(strip_list(make_list(parse_obj.smileys)))
    emojis.append(strip_list(make_list(parse_obj.emojis)))
    urls.append(strip_list(make_list(parse_obj.urls)))
    mentions.append(strip_list(make_list(parse_obj.mentions)))
    numbers.append(strip_list(make_list(parse_obj.numbers)))
    reserveds.append(strip_list(make_list(parse_obj.reserved)))

    if is_hindi == 0:
      raw_tweet_texts.append(tweet_proc.clean(line[1].replace("\n", " ")))
    else:
      raw_tweet_texts.append(hindi_clean(line[1].replace("\n", " "), parse_obj))

print("Number of Datapoints: " + str(datapoints_count))
    

Number of Datapoints: 2963


In [11]:
# Viewing Created Dataset

print("Tweets:")
print(tweets[0: 5])

print("Raw Texts:")
print(raw_tweet_texts[0: 5])

print("Hashtags:")
print(hashtags[0: 5])

print("Smileys:")
print(smileys[0: 5])

print("Emojis:")
print(emojis[0: 5])

print("Urls:")
print(urls[0: 5])

print("Mentions:")
print(mentions[0: 5])

print("Numbers:")
print(numbers[0: 5])

print("Reserved Words:")
print(reserveds[0: 5])

print("Task Labels:")
print(task_1_labels[0: 5])
print(task_2_labels[0: 5])

Tweets:
['1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो 17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम करवाने वाला भारत रत्न कैसे हो सके है?', 'RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ सामने आएंगे। दीपेंदर ने साफ कहा, कि भाजपा और…', 'RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ?  ये आज़मगढ़ है और यहाँ से "टोटी चोर"…', 'Yey nina khothani labafazi benu phambili Finish and klaar 🍑🌊', "RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेकर भाजपा पर तंज कसता हुआ दिखे तो इतिहास के झ…"]
Raw Texts:
['आदमीं को मारने पर गोडसे आतंकी हो सके है तो सिखो ,  भोपाली ,  तमिलों का कत्लेआम करवाने वाला भारत रत्न कैसे हो सके है?', ':  जवाहर यादव ,  अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ सामने आएंगे। दीपेंदर ने साफ कहा ,  कि भाजपा और…', ':  #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं ?  ये आज़मगढ़ है और यहाँ से "टोटी चोर"…

In [12]:
# Generating Emoji Texts
emoji_texts = []

for emo_list in emojis:
  texts = []
  for emoji in emo_list:
    text = emotext(emoji)
    texts.append(text.replace("_", " "))
  emoji_texts.append(texts)

print("Emoji Descriptions:")
print(emoji_texts[0: 5])

Emoji Descriptions:
[[], [], [], ['peach', 'water wave'], []]


In [13]:
# Segmenting Hashtags
segmented_hashtags = []

for hashset in hashtags:
  segmented_set = []
  for tag in hashset:
    word = tag[1: ]
    # removing the hash symbol
    segmented_set.append(seg_tw.segment(word))
  segmented_hashtags.append(segmented_set)

print("Segmented Hashtags:")
print(segmented_hashtags[0: 5])

Segmented Hashtags:
[[], [], ['भगव'], [], []]


In [14]:
name = 'hi.pickle'
dickie = {}
dickie['tweet_id'] = tweet_ids
dickie['task_1'] = task_1_labels
dickie['task_2'] = task_2_labels
dickie['hasoc_id'] = hasoc_ID
dickie['full_tweet'] = tweets
dickie['tweet_raw_text'] = raw_tweet_texts
dickie['hashtags'] = hashtags
dickie['smiley'] = smileys
dickie['emoji'] = emojis
dickie['url'] = urls
dickie['mentions'] = mentions
dickie['numerals'] = numbers
dickie['reserved_word'] = reserveds
dickie['emotext'] = emoji_texts
dickie['segmented_hash'] = segmented_hashtags

with open(name, 'wb') as f:
  pickle.dump(dickie, f)

In [15]:
with open(name, 'rb') as f:
  try_dict = pickle.load(f)

sizes = []
for key in try_dict.keys():
  sizes.append(len(try_dict[key]))

# Verifying if all sizes are equal
print(sizes)

[2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963, 2963]


In [16]:
# ^_^ Thank You